In [ ]:
import requests
import os
import cv2
import base64
from IPython.display import Image, display
import numpy as np
from openai import OpenAI
import time

# Initializing OpenAI client - see https://platform.openai.com/docs/quickstart?context=python
client = OpenAI()

In [1]:
!pwd


/Users/ivanivanov/openai


In [ ]:
api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
video_path = '/Users/ivanivanov/strange_clips/2022-12-20-AMC04207BFA8AB3D2B-01.08.31-01.09.12.mp4'

In [ ]:
video = cv2.VideoCapture(video_path)

base64Frames = []
frame_index = 0
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    if frame_index == 0 or frame_index % 80 == 0:
#     if 1:
        original_height, original_width = frame.shape[:2]
#         resized_frame = cv2.resize(frame, (original_width // 6, original_height // 6))
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
    frame_index += 1

video.release()
print(len(base64Frames), "frames read.")

In [ ]:
display_handle = display(None, display_id=True)
for img in base64Frames:
    display_handle.update(Image(data=base64.b64decode(img.encode("utf-8"))))
    time.sleep(0.025)

In [ ]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "Give me and short description about how much are persons and where they are going. Brown door is entrance of building, black doors are property entrances, wich entrance is used left or right?",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames),
        ],
    },
]
params = {
    "model": "gpt-4o",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 200,
}

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)

In [ ]:
''' "model": "gpt-4-vision-preview",'''